# Import data and libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
caminho_arquivo = '../data/Metro_Interstate_Traffic_Volume.csv'
df = pd.read_csv(caminho_arquivo)

# Basic informations about the dataset

## Head (First Five Rows)

In [2]:
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


## Tail (Last Five Rows)

In [3]:
df.tail()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
48199,NaN,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,3543
48200,NaN,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2781
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,1450
48203,NaN,282.12,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 23:00:00,954


Please note that observations are made on an hourly basis, capturing real-time details such as temperature, precipitation, and weather conditions, with the primary metric being traffic volume.

## Shape (number of rows and columns)

In [4]:
print("\nDataset dimensions (rows, columns):")
df.shape


Dataset dimensions (rows, columns):


(48204, 9)

## Technical Informations

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              61 non-null     object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
 8   traffic_volume       48204 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB


Note that the 'holiday' column only contains 61 non-null values because only a minority of days are holidays.  

## Analysis of Temporal Gaps

In [7]:
# --- Prepare data of date time ---

# Convert the column 'date_time' to the datetime format of pandas
# Essential for any time series analysis
df['date_time'] = pd.to_datetime(df['date_time'])

# Set the 'date_time' column as the index of the DataFrame to make it easier to work with time series data
df = df.set_index('date_time')

# Sort the index to ensure the data is in chronological order
df = df.sort_index()

# Show the start and end of the data to confirm the period
print(f"Start date: {df.index.min()}")
print(f"End date: {df.index.max()}")
print(f"Total records: {len(df)}")


Start date: 2012-10-02 09:00:00
End date: 2018-09-30 23:00:00
Total records: 48204


So, observing the start and end date of the observations, we see that, in fact, the data was collected from October 2012 to September 2018.

## Removing the duplicates 

In [15]:
# Check how many duplicates exist
print(f"Number of duplicate rows (by index): {df.index.duplicated().sum()}")

# --- Handle Conflicting Duplicate Timestamps ---

# 1. Define which columns are numeric and which are categorical
#    (Note: 'traffic_volume' is treated as numeric for aggregation)
numeric_cols = ['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'traffic_volume']
categorical_cols = ['holiday', 'weather_main', 'weather_description']

# 2. Create a dictionary of aggregation rules
agg_rules = {}

# Rule for numeric: take the mean
for col in numeric_cols:
    agg_rules[col] = 'mean'

# Rule for categorical: take the first occurrence
for col in categorical_cols:
    agg_rules[col] = 'first'

print("Aggregation rules defined:")
print(agg_rules)
print("-" * 60)

# 3. Group by the index (timestamp) and apply the aggregation rules
#    This will merge all rows with the same timestamp into one.
df_aggregated = df.groupby(df.index).agg(agg_rules)

# 4. Check the results
print(f"Original number of rows: {len(df)}")
print(f"Number of rows after aggregation: {len(df_aggregated)}")
print(f"Number of duplicate timestamps now: {df_aggregated.index.duplicated().sum()}")

# Now, use this df_aggregated for your analysis
df = df_aggregated.copy()

Number of duplicate rows (by index): 7629
Aggregation rules defined:
{'temp': 'mean', 'rain_1h': 'mean', 'snow_1h': 'mean', 'clouds_all': 'mean', 'traffic_volume': 'mean', 'holiday': 'first', 'weather_main': 'first', 'weather_description': 'first'}
------------------------------------------------------------
Original number of rows: 48204
Number of rows after aggregation: 40575
Number of duplicate timestamps now: 0


## Handle Missing Values

In [16]:

# --- Find the missing values ---

# 1. Create an ideal date index, with hourly frequency ('h'),
#    from the first date to the last date recorded in your dataset.
start_date = df.index.min()
end_date = df.index.max()

# 'h' means 'hourly' frequency
ideal_index = pd.date_range(start=start_date, end=end_date, freq='h')

# Print a summary comparing the ideal index with the actual DataFrame index
print(f"Expected records (hourly): {len(ideal_index)}")
print(f"Existing records: {len(df)}")
print(f"Total missing hours: {len(ideal_index) - len(df)}")
print("-" * 60)

# 2. Find out which dates/times are in the ideal index but not in the index of your DataFrame
missing_dates = ideal_index.difference(df.index)

if missing_dates.empty:
    print("No missing time intervals found!")
else:
    print("MISSING FACTS: The following time intervals are missing from the dataset:")
    print(missing_dates)

print(f"\nTotal missing time intervals: {len(missing_dates)}")


Expected records (hourly): 52551
Existing records: 40575
Total missing hours: 11976
------------------------------------------------------------
MISSING FACTS: The following time intervals are missing from the dataset:
DatetimeIndex(['2012-10-03 07:00:00', '2012-10-03 10:00:00',
               '2012-10-03 11:00:00', '2012-10-03 17:00:00',
               '2012-10-05 02:00:00', '2012-10-05 04:00:00',
               '2012-10-06 03:00:00', '2012-10-07 01:00:00',
               '2012-10-07 02:00:00', '2012-10-09 03:00:00',
               ...
               '2018-03-24 05:00:00', '2018-03-24 06:00:00',
               '2018-03-24 07:00:00', '2018-03-29 02:00:00',
               '2018-05-05 02:00:00', '2018-06-02 02:00:00',
               '2018-08-07 07:00:00', '2018-08-07 08:00:00',
               '2018-08-07 09:00:00', '2018-08-23 02:00:00'],
              dtype='datetime64[ns]', length=11976, freq=None)

Total missing time intervals: 11976
